https://www.kaggle.com/uysimty/keras-cnn-dog-or-cat-classification
https://www.kaggle.com/uysimty/get-start-image-classification/notebook

In [1]:
import numpy as np
import pandas as pd 
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
print(os.listdir("drive/My Drive/LUNG CANCER DETECTION"))

In [ ]:
filenames = os.listdir("drive/My Drive/LUNG CANCER DETECTION/Patch_old")
import pandas as pd
categories = pd.read_csv('drive/My Drive/LUNG CANCER DETECTION/patch_labels.csv',header=None)

In [ ]:
pip install natsort

In [ ]:
import natsort #sort the filenames file
filenames=natsort.natsorted(filenames)

In [ ]:
print(categories)

In [ ]:
# category.iloc[58,0]
# category.iloc[59,0]
categories.iloc[59,0]

In [ ]:
import re
def index(str):
  match = re.match(r"([a-zA-Z]+)([0-9]+)", str, re.I)
  if match:
    items = match.groups()
  return int(items[1])-1; 
print(index("asd123.tiff"))

In [ ]:
labels=[]
for filename in filenames:
    category = categories.iloc[index(filename),0]
    labels.append(category)



In [ ]:
df = pd.DataFrame({
    'filename': filenames,
    'category': labels
})

In [ ]:
df

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df['category'].value_counts().plot.bar()

In [ ]:
import cv2
sample = random.choice(filenames)
img = cv2.imread("drive/My Drive/LUNG CANCER DETECTION/Patch_old/"+sample) #here img is numpy array
dimensions = img.shape
print(dimensions) 
plt.imshow(img)

In [ ]:
IMAGE_WIDTH=61;
IMAGE_HEIGHT=61;
IMAGE_CHANNELS=3;
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

model = Sequential()
model.add(Conv2D(32, (2, 2), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax')) # 2 because we have cat and dog classes

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience=10)

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
callbacks = [earlystop, learning_rate_reduction]

In [ ]:
df["category"] = df["category"].replace({0: 'YES', 1: 'NO'}) 

In [ ]:
df.head()

In [ ]:
train_df, validate_df = train_test_split(df, test_size=0.20, random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [ ]:
train_df['category'].value_counts().plot.bar()

In [ ]:
validate_df['category'].value_counts().plot.bar()

In [ ]:
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
batch_size=15
print(total_train)
print(total_validate)

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    "drive/My Drive/LUNG CANCER DETECTION/Patch_old/", 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "drive/My Drive/LUNG CANCER DETECTION/NEW DATA/Patch_old/", 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

In [ ]:
#fit model
FAST_RUN=False
epochs=3 if FAST_RUN else 50
history = model.fit_generator(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)

In [ ]:
#accuracy =63.16%
#save model
model.save_weights("model.h5")

In [ ]:
#visualize training
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
ax1.plot(history.history['loss'], color='b', label="Training loss")
ax1.plot(history.history['val_loss'], color='r', label="validation loss")
ax1.set_xticks(np.arange(1, epochs, 1))
ax1.set_yticks(np.arange(0, 1, 0.1))

ax2.plot(history.history['acc'], color='b', label="Training accuracy")
ax2.plot(history.history['val_acc'], color='r',label="Validation accuracy")
ax2.set_xticks(np.arange(1, epochs, 1))

legend = plt.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()